In [ ]:
# File: woven.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import random

import numpy as np
from frontend import App


def make_woven_cylinder(n: int, offset: float, scale: float):
    dx, width = 1.0 / (n - 1), 1.25
    scale = 2.0 * 1.48 * scale
    v_steps = int(25.0 * scale)
    sep, strands = 0.5, []

    for i in range(v_steps):
        theta = 2.0 * np.pi * i / v_steps
        xyz = np.zeros((n, 3))
        xyz[:, 0] = width * (2.0 * dx * np.arange(n) - 1.0)
        xyz[:, 1], xyz[:, 2] = sep * np.sin(theta), sep * np.cos(theta)
        strands.append((xyz, False))

    h_steps = int(30.0 * scale)
    ring_steps = v_steps * 3
    assert ring_steps % 2 == 0, "ring_steps must be even"
    amp, dx_h, half_v = 1.2 * offset, 1.0 / (h_steps - 1), v_steps // 2

    for i in range(1, h_steps - 1):
        sgn = 1.0 if (i % 2 == 0) else -1.0
        xyz = np.zeros((ring_steps, 3))
        xyz[:, 0] = width * (2.0 * dx_h * i - 1.0)
        j_indices = np.arange(ring_steps)
        theta_vals = 2.0 * np.pi * j_indices / ring_steps
        r = sep + sgn * amp * np.cos(half_v * theta_vals)
        xyz[:, 1], xyz[:, 2] = r * np.sin(theta_vals), r * np.cos(theta_vals)
        strands.append((xyz, True))

    return strands


app = App.create("woven")
scene = app.scene.create()

k = 0
angular_vel, move_delta, t_end = 360 + 10 * random.random(), 0.15, 10
for V, closed in make_woven_cylinder(256, 4e-3, 2.0):
    E = [[i, i + 1] for i in range(len(V) - 1)]
    if closed:
        E.append([len(V) - 1, 0])
    name = f"strand-{k}"
    app.asset.add.rod(name, V, np.array(E, dtype=np.uint32))
    obj = scene.add(name)
    (
        obj.param.set("bend", 1e-3)
        .set("young-mod", 1e5)
        .set("contact-gap", 1.5e-3)
        .set("contact-offset", 2e-3)
        .set("friction", 0.01)
        .set("length-factor", 0.8)
    )
    if not closed:
        (
            obj.pin(obj.grab([-1, 0, 0]))
            .spin(axis=[1, 0, 0], angular_velocity=angular_vel)
            .move_by([move_delta, 0, 0], t_end)
        )
        (
            obj.pin(obj.grab([1, 0, 0]))
            .spin(axis=[-1, 0, 0], angular_velocity=angular_vel)
            .move_by([-move_delta, 0, 0], t_end)
        )
    k += 1

scene = scene.build().report()
scene.preview()

In [ ]:
session = app.session.create(scene)
(
    session.param.set("frames", 450)
    .set("dt", 1e-2)
    .set("gravity", 0.0)
    .set("isotropic-air-friction", 1e-3)
)
session = session.build()

In [ ]:
session.start().preview()
session.stream()

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()